# Data preparation

In [2]:
import glob
import os
import re
import numpy as np         
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [3]:
txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/song_lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['The Cat in the Hat', 'By Dr. Seuss', 'The sun did not shine.']


# Preprocessing

In [4]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.v

<start> this is sample sentence . <end>


In [5]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
        
    corpus.append(preprocess_sentence(sentence))
        
corpus[:10]

['<start> the cat in the hat <end>',
 '<start> by dr . seuss <end>',
 '<start> the sun did not shine . <end>',
 '<start> it was too wet to play . <end>',
 '<start> so we sat in the house <end>',
 '<start> all that cold cold wet day . <end>',
 '<start> i sat there with sally . <end>',
 '<start> we sat there we two . <end>',
 '<start> and i said how i wish <end>',
 '<start> we had something to do ! <end>']

In [6]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.
    total_data_text = list(tensor)
    num_tokens = [len(tokens) for tokens in total_data_text]
    max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
    maxlen = int(max_tokens)

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post',maxlen=maxlen)
    # maxlen=maxlen을 통한 outlier 제거

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2    6  903 ...    0    0    0]
 [   2  122 2584 ...    0    0    0]
 [   2    6  304 ...    0    0    0]
 ...
 [   2  673   27 ...    0    0    0]
 [   2  673   27 ...    0    0    0]
 [   2  673   27 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fc0f558ab10>


In [7]:
# tokenizer에 구축된 단어 사전의 인덱스 확인
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [8]:
X = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <END>가 아니라 <pad>일 가능성이 높습니다.
y = tensor[:, 1:]    # tensor에서 <START>를 잘라내서 타겟 문장을 생성합니다.

print(X[0])
print(y[0])
print(X[0].shape) # max length  = 346

[   2    6  903   14    6 1350    3    0    0    0    0    0    0    0
    0    0    0    0    0]
[   6  903   14    6 1350    3    0    0    0    0    0    0    0    0
    0    0    0    0    0]
(19,)


In [9]:
enc_train, enc_val, dec_train, dec_val = train_test_split(X, y, test_size=0.2, shuffle = True, random_state=42)

In [10]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (140599, 19)
Target Train: (140599, 19)


In [15]:
BUFFER_SIZE = len(X)
BATCH_SIZE = 256
steps_per_epoch = len(X) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개

dataset = tf.data.Dataset.from_tensor_slices((X, y)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 19), (256, 19)), types: (tf.int32, tf.int32)>

# 모델

In [12]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [16]:
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 19, 12001), dtype=float32, numpy=
array([[[-1.83450422e-04,  1.54117326e-04, -1.03245788e-04, ...,
         -2.18200730e-05,  6.39171121e-05,  1.51997796e-04],
        [-3.83957959e-04,  2.32917737e-04,  1.16457115e-04, ...,
          5.56566047e-05,  1.45285652e-04,  1.28228552e-04],
        [-5.86198701e-04,  1.63399876e-04,  2.09341801e-04, ...,
         -3.59306500e-06,  1.66437007e-04,  1.73283945e-04],
        ...,
        [-3.42818815e-03, -7.92458130e-04, -1.93423603e-03, ...,
          2.96862028e-03, -3.18909879e-03,  6.90072367e-04],
        [-3.39134317e-03, -8.19502340e-04, -1.98594737e-03, ...,
          3.06102703e-03, -3.29980394e-03,  7.52031046e-04],
        [-3.33969900e-03, -8.29085824e-04, -2.02124752e-03, ...,
          3.13235889e-03, -3.38933012e-03,  8.21006543e-04]],

       [[-1.83450422e-04,  1.54117326e-04, -1.03245788e-04, ...,
         -2.18200730e-05,  6.39171121e-05,  1.51997796e-04],
        [-2.19654437e-04,  1.65621561e-04, -1

In [17]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [19]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
686/686 [==============================] - 139s 203ms/step - loss: 2.6818
Epoch 2/10
686/686 [==============================] - 138s 202ms/step - loss: 2.3123
Epoch 3/10
686/686 [==============================] - 133s 195ms/step - loss: 2.1211
Epoch 4/10
686/686 [==============================] - 145s 212ms/step - loss: 1.9613
Epoch 5/10
686/686 [==============================] - 144s 210ms/step - loss: 1.8236
Epoch 6/10
686/686 [==============================] - 146s 212ms/step - loss: 1.7030
Epoch 7/10
686/686 [==============================] - 146s 212ms/step - loss: 1.5957
Epoch 8/10
686/686 [==============================] - 146s 212ms/step - loss: 1.4985
Epoch 9/10
686/686 [==============================] - 146s 212ms/step - loss: 1.4094
Epoch 10/10
686/686 [==============================] - 146s 212ms/step - loss: 1.3256


NameError: name 'epochs' is not defined

In [ ]:
history = model.fit(enc_train, 
          dec_train, 
          epochs=10,
          batch_size=256,
          validation_data=(enc_val, dec_val),
          verbose=1)

In [21]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
																 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <END>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [23]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , liberian girl <end> '